## Since we are going to fine tune a LLM, we need a system with GPU.
We will be using Google Colab for this purpose.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [6]:
!pip install transformers
!pip install torch


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/56/7e/67c3fe2b8c33f40af06326a3d6ae7776b3e3a01daa8f71d125d78594d874/torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/a3/dc/17031897dae0efacfea57dfd3a82fdd2a2aeb58e0ff71b77b87e44edc772/setuptools-80.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/eb/8d/776adee7bbf76365fdd7f2552710282c79a4ead5d2a46408c9043a2b70ba/networkx-3.5-py3-none-any.whl.metadata
  Obtaining dependency information for jinja2 from https://fil

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

try:
    os.environ["HF_TOKEN"]
except KeyError:
    print("Token not found!")

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/workspaces/Concepts-to-deploy-GEN-AI-projects/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
tokenizer("Hello world")

{'input_ids': [2, 9259, 1902], 'attention_mask': [1, 1, 1]}

In [10]:
input_conversation = [
    {"role": "user", "content": "Which is the best place to learn GenAI"},
    {"role": "assistant", "content": "The best place to learn AI is"}
]

In [12]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False
)
input_tokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is<end_of_turn>\n'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to(device)